## Q1. Conversion Funnel Deep Dive
​Compute conversion rates across funnel stages ​(Search → Quote → Booking → Completed). Segment by time-of-day Morning (6–10 AM), Day (10 AM–5 PM), Evening (5–9 PM), Night (9 PM–6 AM) and trip length (short <5 km, medium 5–15 km, long >15 km), Identify the maximum drop within each segment. Propose hypotheses for observed drops and validate with data

In [1]:
import os
import pandas as pd
import pytz
import matplotlib.pyplot as plt
import seaborn as sns

folder_path = '/content/drive/My Drive/NY Datathon 2025/NY Datathon 2025'

if os.path.exists(folder_path):
    files = os.listdir(folder_path)
    print(f"Files in '{folder_path}':")
    for file in files:
        print(file)
else:
    print(f"Folder '{folder_path}' not found.")

csv_files = []
for file in files:
    if file.endswith('.csv'):
        csv_files.append(file)

print("CSV files found:")
for csv_file in csv_files:
    print(csv_file)

dfs = {}

for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df_name = os.path.splitext(csv_file)[0]
    dfs[df_name] = pd.read_csv(file_path)

for df_name, df in dfs.items():
    print(f"DataFrame '{df_name}':")
    display(df.head())

def convert_utc_to_ist(utc_timestamp):

    if pd.isna(utc_timestamp):
        return None
    try:
        utc_time = pd.to_datetime(utc_timestamp, utc=True)
        ist_time = utc_time.tz_convert('Asia/Kolkata')
        return ist_time
    except Exception as e:
        print(f"Error converting timestamp {utc_timestamp}: {e}")
        return None

for df_name, time_col in [
    ('search_data', 'search_request_created_at'),
    ('quote_data', 'quote_created_at'),
    ('booking_data', 'booking_created_at'),
    ('booking_cancellation_data', 'cancelled_at')
]:
    dfs[df_name][f'{time_col}_ist'] = dfs[df_name][time_col].apply(convert_utc_to_ist)


for df_name, time_col in [
    ('search_data', 'search_request_created_at'),
    ('quote_data', 'quote_created_at'),
    ('booking_data', 'booking_created_at')
]:
    dfs[df_name][time_col] = pd.to_datetime(dfs[df_name][time_col])

merged_df = (
    dfs['search_data']
    .merge(dfs['quote_data'], on='search_request_id', how='left')
    .merge(dfs['booking_data'], on='quote_id', how='left')
    .merge(dfs['booking_cancellation_data'], on='booking_id', how='left')
)




merged_df['hour_of_day'] = merged_df['search_request_created_at'].dt.hour

def get_time_of_day_segment(hour):
    if 4 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 16:
        return 'Day'
    elif 16 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

merged_df['time_of_day_segment'] = merged_df['hour_of_day'].apply(get_time_of_day_segment)


def categorize_trip_length(distance):
    if pd.isna(distance):
        return 'unknown'
    elif distance < 5000:
        return 'short'
    elif 5000 <= distance < 20000:
        return 'medium'
    else:
        return 'long'

merged_df['trip_length_segment'] = merged_df['estimated_distance'].apply(categorize_trip_length)
print("\n trip length distribution:")
display(merged_df['trip_length_segment'].value_counts())


funnel_stages = ['search_request_id', 'quote_id', 'booking_id', 'status']

funnel_counts = (
    merged_df.groupby(['time_of_day_segment', 'trip_length_segment'])[funnel_stages]
    .count()
    .rename(columns={
        'search_request_id': 'Search',
        'quote_id': 'Quote',
        'booking_id': 'Booking',
        'status': 'Completed'
    })
)

completed_counts = (
    merged_df[merged_df['status'] == 'COMPLETED']
    .groupby(['time_of_day_segment', 'trip_length_segment'])['booking_id']
    .count()
)

funnel_counts['Completed'] = completed_counts.fillna(0).astype(int)

funnel_counts['Search_to_Quote_Conversion'] = (funnel_counts['Quote'] / funnel_counts['Search']) * 100
funnel_counts['Quote_to_Booking_Conversion'] = (funnel_counts['Booking'] / funnel_counts['Quote']) * 100
funnel_counts['Booking_to_Completed_Conversion'] = (funnel_counts['Completed'] / funnel_counts['Booking']) * 100

print("\n📊 Funnel conversion rates per segment:")
display(funnel_counts)


max_drop_per_segment = {}
for index, row in funnel_counts.iterrows():
    conversions = {
        'Search_to_Quote_Conversion': row['Search_to_Quote_Conversion'],
        'Quote_to_Booking_Conversion': row['Quote_to_Booking_Conversion'],
        'Booking_to_Completed_Conversion': row['Booking_to_Completed_Conversion']
    }
    max_drop_stage = min(conversions, key=conversions.get)
    max_drop_per_segment[index] = {
        'Stage': max_drop_stage,
        'Max_Drop_Percentage': conversions[max_drop_stage]
    }

max_drop_df = pd.DataFrame.from_dict(max_drop_per_segment, orient='index')
print("\n🔻 Maximum conversion drop per segment:")
display(max_drop_df)


print("\n Estimated Distance & Duration vs Quote Analysis:")


merged_df['got_quote'] = merged_df['quote_id'].notna()

plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
sns.boxplot(x='got_quote', y='estimated_distance', data=merged_df)
plt.title('Estimated Distance vs. Got Quote')
plt.xlabel('Got Quote')
plt.ylabel('Estimated Distance')

plt.subplot(1, 2, 2)
sns.boxplot(x='got_quote', y='estimated_duration', data=merged_df)
plt.title('Estimated Duration vs. Got Quote')
plt.xlabel('Got Quote')
plt.ylabel('Estimated Duration')

plt.tight_layout()
plt.show()


quoted_df = merged_df[merged_df['got_quote'] == True].copy()

print("\nAn alysis of Fare and Pickup Metrics for Quoted Searches:")

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
sns.boxplot(x='time_of_day_segment', y='estimated_fare', data=quoted_df)
plt.title('Estimated Fare by Time of Day')
plt.xlabel('Time of Day')
plt.ylabel('Estimated Fare')

plt.subplot(1, 3, 2)
sns.kdeplot(quoted_df['distance_to_pickup'], fill=True)
plt.title('Distance to Pickup Distribution')
plt.xlabel('Distance to Pickup')

plt.subplot(1, 3, 3)
sns.kdeplot(quoted_df['duration_to_pickup'], fill=True)
plt.title('Duration to Pickup Distribution')
plt.xlabel('Duration to Pickup')

plt.tight_layout()
plt.show()

print("\n  Final Funnel Conversion Summary:")
display(funnel_counts)

print("\n Maximum Conversion Drop Summary:")
display(max_drop_df)

Folder '/content/drive/My Drive/NY Datathon 2025/NY Datathon 2025' not found.


NameError: name 'files' is not defined

## Q2. Booking Cancellations
Compute the overall cancellation rate. Break down by driver vs rider cancellations, pickup distance buckets and trip distance buckets. Identify the top 3 driver cancellation reasons and analyze their trends over time-of-day and trip length. Analyze the relationship between driver ratings and driver cancellations, and provide key insights​

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os

folder_path = '/content/drive/My Drive/NY Datathon 2025/NY Datathon 2025'

if os.path.exists(folder_path):
    files = os.listdir(folder_path)
    print(f"Files in '{folder_path}':")
    for file in files:
        print(file)
else:
    print(f"Folder '{folder_path}' not found.")


csv_files = []
for file in files:
    if file.endswith('.csv'):
        csv_files.append(file)

print("CSV files found:")
for csv_file in csv_files:
    print(csv_file)

import pandas as pd

dfs = {}

for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df_name = os.path.splitext(csv_file)[0]
    dfs[df_name] = pd.read_csv(file_path)

for df_name, df in dfs.items():
    print(f"DataFrame '{df_name}':")
    display(df.head())
  import pandas as pd

def convert_utc_to_ist(utc_timestamp):

    if pd.isna(utc_timestamp):
        return None
    try:
        utc_time = pd.to_datetime(utc_timestamp, utc=True)
        ist_time = utc_time.tz_convert('Asia/Kolkata')
        return ist_time
    except Exception as e:
        print(f"Error converting timestamp {utc_timestamp}: {e}")
        return None

dfs['search_data']['search_request_created_at_ist'] = dfs['search_data']['search_request_created_at'].apply(convert_utc_to_ist)
dfs['quote_data']['quote_created_at_ist'] = dfs['quote_data']['quote_created_at'].apply(convert_utc_to_ist)
dfs['booking_data']['booking_created_at_ist'] = dfs['booking_data']['booking_created_at'].apply(convert_utc_to_ist)
dfs['booking_cancellation_data']['cancelled_at_ist'] = dfs['booking_cancellation_data']['cancelled_at'].apply(convert_utc_to_ist)


print("DataFrame 'search_data' with IST timestamp:")
display(dfs['search_data'].head())

print("DataFrame 'quote_data' with IST timestamp:")
display(dfs['quote_data'].head())

print("DataFrame 'booking_data' with IST timestamp:")
display(dfs['booking_data'].head())

print("DataFrame 'booking_cancellation_data' with IST timestamp:")
display(dfs['booking_cancellation_data'].head())
total_bookings = dfs['booking_data'].shape[0]
total_cancellations = dfs['booking_cancellation_data'].shape[0]

print(f"Total number of bookings: {total_bookings}")
print(f"Total number of cancellations: {total_cancellations}")
print("Columns in dfs['search_data']:")
print(dfs['search_data'].columns)

cancelled_bookings_with_quote = pd.merge(merged_cancellations, dfs['quote_data'], on='quote_id', how='left')

cancelled_bookings_with_search = pd.merge(cancelled_bookings_with_quote, dfs['search_data'], on='search_request_id', how='left')

trip_distance_bins = [0, 5000, 10000, 25000, 50000, 100000, float('inf')]
trip_distance_labels = ['0-5km', '5-10km', '10-25km', '25-50km', '50-100km', '>100km']

cancelled_bookings_with_search['trip_distance_bucket'] = pd.cut(cancelled_bookings_with_search['estimated_distance'], bins=trip_distance_bins, labels=trip_distance_labels, right=False)

cancellations_by_trip_distance = cancelled_bookings_with_search['trip_distance_bucket'].value_counts().reset_index()
cancellations_by_trip_distance.columns = ['trip_distance_bucket', 'cancellation_count']

all_bookings_with_quote = pd.merge(dfs['booking_data'], dfs['quote_data'], on='quote_id', how='left')
all_bookings_with_search = pd.merge(all_bookings_with_quote, dfs['search_data'], on='search_request_id', how='left')
all_bookings_with_search['trip_distance_bucket'] = pd.cut(all_bookings_with_search['estimated_distance'], bins=trip_distance_bins, labels=trip_distance_labels, right=False)
total_bookings_by_trip_distance = all_bookings_with_search['trip_distance_bucket'].value_counts().reset_index()
total_bookings_by_trip_distance.columns = ['trip_distance_bucket', 'total_booking_count']

trip_cancellation_rates = pd.merge(cancellations_by_trip_distance, total_bookings_by_trip_distance, on='trip_distance_bucket', how='left')

trip_cancellation_rates['cancellation_rate'] = (trip_cancellation_rates['cancellation_count'] / trip_cancellation_rates['total_booking_count']) * 100

print("\nCancellation rate by trip distance bucket:")
display(trip_cancellation_rates.sort_values('trip_distance_bucket'))
driver_cancellations = merged_cancellations[merged_cancellations['source'] == 'ByDriver']

driver_cancellation_reasons = driver_cancellations['reason_code'].value_counts()

top_3_driver_cancellation_reasons = driver_cancellation_reasons.head(3)

print("Top 3 Driver Cancellation Reasons:")
display(top_3_driver_cancellation_reasons)
import matplotlib.pyplot as plt

top_reasons = top_3_driver_cancellation_reasons.index.tolist()
filtered_cancellations = merged_cancellations[merged_cancellations['reason_code'].isin(top_reasons)]
filtered_cancellations['cancelled_at'] = pd.to_datetime(filtered_cancellations['cancelled_at'])

filtered_cancellations['cancellation_hour'] = filtered_cancellations['cancelled_at'].dt.hour

cancellation_trends = filtered_cancellations.groupby(['cancellation_hour', 'reason_code']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
for reason in top_reasons:
    plt.plot(cancellation_trends.index, cancellation_trends[reason], marker='o', linestyle='-', label=reason)

plt.title('Trends of Top 3 Driver Cancellation Reasons Over Time of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Cancellations')
plt.xticks(range(24))
plt.legend()
plt.grid(True)
plt.show()
import matplotlib.pyplot as plt

driver_cancellations_with_quote = pd.merge(driver_cancellations, dfs['quote_data'], on='quote_id', how='left')

driver_cancellations_with_search = pd.merge(driver_cancellations_with_quote, dfs['search_data'], on='search_request_id', how='left')

trip_distance_bins = [0, 5000, 10000, 25000, 50000, 100000, float('inf')]
trip_distance_labels = ['0-5km', '5-10km', '10-25km', '25-50km', '50-100km', '>100km']

driver_cancellations_with_search['trip_distance_bucket'] = pd.cut(driver_cancellations_with_search['estimated_distance'], bins=trip_distance_bins, labels=trip_distance_labels, right=False)

top_reasons = top_3_driver_cancellation_reasons.index.tolist()
filtered_driver_cancellations = driver_cancellations_with_search[driver_cancellations_with_search['reason_code'].isin(top_reasons)]

cancellation_trends_by_distance = filtered_driver_cancellations.groupby(['trip_distance_bucket', 'reason_code']).size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
cancellation_trends_by_distance.plot(kind='bar', figsize=(12, 6))

plt.title('Trends of Top 3 Driver Cancellation Reasons by Trip Length')
plt.xlabel('Trip Distance Bucket')
plt.ylabel('Number of Cancellations')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Reason Code')
plt.tight_layout()
plt.show()
print("Non-numeric values in driver_cancellations_with_rating['driver_rating']:")
display(driver_cancellations_with_rating[pd.to_numeric(driver_cancellations_with_rating['driver_rating'], errors='coerce').isna()]['driver_rating'].unique())

print("\nNon-numeric values in dfs['quote_data']['driver_rating']:")
display(dfs['quote_data'][pd.to_numeric(dfs['quote_data']['driver_rating'], errors='coerce').isna()]['driver_rating'].unique())

driver_cancellations_with_rating['driver_rating'] = pd.to_numeric(driver_cancellations_with_rating['driver_rating'], errors='coerce')
dfs['quote_data']['driver_rating'] = pd.to_numeric(dfs['quote_data']['driver_rating'], errors='coerce')

driver_cancellations_with_rating.dropna(subset=['driver_rating'], inplace=True)
dfs['quote_data'].dropna(subset=['driver_rating'], inplace=True)

rating_bins = [0, 4, 4.5, 4.8, 5.1]
rating_labels = ['<4', '4-4.5', '4.5-4.8', '4.8-5']


driver_cancellations_with_rating['driver_rating_bucket'] = pd.cut(driver_cancellations_with_rating['driver_rating'], bins=rating_bins, labels=rating_labels, right=False, include_lowest=True)

cancellations_by_rating = driver_cancellations_with_rating['driver_rating_bucket'].value_counts().reset_index()
cancellations_by_rating.columns = ['driver_rating_bucket', 'cancellation_count']

dfs['quote_data']['driver_rating_bucket'] = pd.cut(dfs['quote_data']['driver_rating'], bins=rating_bins, labels=rating_labels, right=False, include_lowest=True)

total_quotes_by_rating = dfs['quote_data']['driver_rating_bucket'].value_counts().reset_index()
total_quotes_by_rating.columns = ['driver_rating_bucket', 'total_quote_count']

rating_cancellation_rates = pd.merge(cancellations_by_rating, total_quotes_by_rating, on='driver_rating_bucket', how='left')

rating_cancellation_rates['cancellation_rate'] = (rating_cancellation_rates['cancellation_count'] / rating_cancellation_rates['total_quote_count']) * 100

print("\nDriver cancellation rate by driver rating bucket:")
display(rating_cancellation_rates.sort_values('driver_rating_bucket'))

# 3rd question
## ​Q3. Critical Stage & Root Cause Analysis Identify the most critical funnel stage using insights from Q1 and Q2. Perform clustering on drivers/riders to discover behavioral segments (e.g., high-quote, low-booking drivers). Analyze weekly time-series trends for conversion and cancellations to detect seasonality.

In [ ]:
num_search_requests = dfs['search_data']['search_request_id'].nunique()
num_quotes = dfs['quote_data']['quote_id'].nunique()
num_bookings = dfs['booking_data']['booking_id'].nunique()

search_to_quote_rate = (num_quotes / num_search_requests) * 100
quote_to_booking_rate = (num_bookings / num_quotes) * 100


print(f"Number of unique search requests: {num_search_requests}")
print(f"Number of unique quotes: {num_quotes}")
print(f"Number of unique bookings: {num_bookings}")
print(f"Conversion rate from Search to Quote: {search_to_quote_rate:.2f}%")
print(f"Conversion rate from Quote to Booking: {quote_to_booking_rate:.2f}%")
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

driver_features = dfs['quote_data'].groupby('driver_id').agg(
    total_quotes=('quote_id', 'count'),
    avg_driver_rating=('driver_rating', 'mean'),
).reset_index()

driver_bookings = dfs['booking_data'].merge(dfs['quote_data'][['quote_id', 'driver_id']], on='quote_id', how='left')
driver_booked_quotes_count = driver_bookings.groupby('driver_id').agg(booked_quotes=('booking_id', 'count')).reset_index()

driver_cancellations = dfs['booking_cancellation_data'].merge(dfs['booking_data'][['booking_id', 'quote_id']], on='booking_id', how='left')
driver_cancelled_quotes_count = driver_cancellations.merge(dfs['quote_data'][['quote_id', 'driver_id']], on='quote_id', how='left').groupby('driver_id').agg(cancelled_quotes=('booking_id', 'count')).reset_index()


driver_features = driver_features.merge(driver_booked_quotes_count, on='driver_id', how='left').merge(driver_cancelled_quotes_count, on='driver_id', how='left')
driver_features.fillna(0, inplace=True)

driver_features['quote_acceptance_rate'] = driver_features['booked_quotes'] / driver_features['total_quotes']
driver_features['cancellation_rate'] = driver_features['cancelled_quotes'] / driver_features['booked_quotes']

rider_features = dfs['booking_data'].groupby('rider_id').agg(
    total_bookings=('booking_id', 'count'),
).reset_index()

rider_cancellations_count = dfs['booking_cancellation_data'][dfs['booking_cancellation_data']['source'] == 'ByUser'].groupby('booking_id').size().reset_index(name='cancellation_count')
rider_cancellations_count = rider_cancellations_count.merge(dfs['booking_data'][['booking_id', 'rider_id']], on='booking_id', how='left')
rider_cancelled_bookings_count = rider_cancellations_count.groupby('rider_id').agg(cancelled_bookings=('booking_id', 'count')).reset_index()

rider_features = rider_features.merge(rider_cancelled_bookings_count, on='rider_id', how='left')
rider_features.fillna(0, inplace=True)

rider_features['cancellation_rate'] = rider_features['cancelled_bookings'] / rider_features['total_bookings']


driver_clustering_features = driver_features[['total_quotes', 'avg_driver_rating', 'quote_acceptance_rate', 'cancellation_rate']]
rider_clustering_features = rider_features[['total_bookings', 'cancellation_rate']]

imputer = SimpleImputer(strategy='mean')
driver_clustering_features_imputed = imputer.fit_transform(driver_clustering_features)
rider_clustering_features_imputed = imputer.fit_transform(rider_clustering_features)


scaler_driver = StandardScaler()
driver_scaled_features = scaler_driver.fit_transform(driver_clustering_features_imputed)

scaler_rider = StandardScaler()
rider_scaled_features = scaler_rider.fit_transform(rider_clustering_features_imputed)

print("Driver features for clustering (first 5 rows after scaling and imputation):")
display(pd.DataFrame(driver_scaled_features, columns=driver_clustering_features.columns).head())

print("\nRider features for clustering (first 5 rows after scaling and imputation):")
display(pd.DataFrame(rider_scaled_features, columns=rider_clustering_features.columns).head())
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

inertia_driver = []
k_range_driver = range(1, 11)
for k in k_range_driver:
    kmeans_driver = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_driver.fit(driver_scaled_features)
    inertia_driver.append(kmeans_driver.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(k_range_driver, inertia_driver, marker='o')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Driver Clustering')
plt.xticks(k_range_driver)
plt.grid(True)
plt.show()

n_clusters_driver = 3
kmeans_driver = KMeans(n_clusters=n_clusters_driver, random_state=42, n_init=10)
driver_features['driver_cluster'] = kmeans_driver.fit_predict(driver_scaled_features)

inertia_rider = []
k_range_rider = range(1, 11)
for k in k_range_rider:
    kmeans_rider = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_rider.fit(rider_scaled_features)
    inertia_rider.append(kmeans_rider.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(k_range_rider, inertia_rider, marker='o')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method for Rider Clustering')
plt.xticks(k_range_rider)
plt.grid(True)
plt.show()

n_clusters_rider = 3
kmeans_rider = KMeans(n_clusters=n_clusters_rider, random_state=42, n_init=10)
rider_features['rider_cluster'] = kmeans_rider.fit_predict(rider_scaled_features)

print("\nDriver features with cluster labels (first 5 rows):")
display(driver_features.head())

print("\nRider features with cluster labels (first 5 rows):")
display(rider_features.head())
print("\nDriver Cluster Analysis (Mean of Features per Cluster):")
display(driver_features.groupby('driver_cluster')[driver_clustering_features.columns].mean())

print("\nRider Cluster Analysis (Mean of Features per Cluster):")
display(rider_features.groupby('rider_cluster')[rider_clustering_features.columns].mean())
dfs['search_data']['search_request_created_at'] = pd.to_datetime(dfs['search_data']['search_request_created_at'])
dfs['quote_data']['quote_created_at'] = pd.to_datetime(dfs['quote_data']['quote_created_at'])
dfs['booking_data']['booking_created_at'] = pd.to_datetime(dfs['booking_data']['booking_created_at'])
dfs['booking_cancellation_data']['cancelled_at'] = pd.to_datetime(dfs['booking_cancellation_data']['cancelled_at'])

weekly_searches = dfs['search_data'].set_index('search_request_created_at').resample('W')['search_request_id'].nunique()
weekly_quotes = dfs['quote_data'].set_index('quote_created_at').resample('W')['quote_id'].nunique()
weekly_bookings = dfs['booking_data'].set_index('booking_created_at').resample('W')['booking_id'].nunique()
weekly_cancellations = dfs['booking_cancellation_data'].set_index('cancelled_at').resample('W')['booking_id'].nunique()

print("Weekly searches (first 5 weeks):")
display(weekly_searches.head())
print("\nWeekly quotes (first 5 weeks):")
display(weekly_quotes.head())
print("\nWeekly bookings (first 5 weeks):")
display(weekly_bookings.head())
print("\nWeekly cancellations (first 5 weeks):")
display(weekly_cancellations.head())
import matplotlib.pyplot as plt

weekly_search_to_quote_rate = (weekly_quotes / weekly_searches) * 100
weekly_quote_to_booking_rate = (weekly_bookings / weekly_quotes) * 100

weekly_cancellation_rate = (weekly_cancellations / weekly_bookings) * 100

plt.figure(figsize=(14, 7))

plt.plot(weekly_search_to_quote_rate.index, weekly_search_to_quote_rate.values, marker='o', linestyle='-', label='Search to Quote Conversion Rate')
plt.plot(weekly_quote_to_booking_rate.index, weekly_quote_to_booking_rate.values, marker='o', linestyle='-', label='Quote to Booking Conversion Rate')
plt.plot(weekly_cancellation_rate.index, weekly_cancellation_rate.values, marker='o', linestyle='-', label='Overall Cancellation Rate')

plt.title('Weekly Trends of Conversion and Cancellation Rates')
plt.xlabel('Week')
plt.ylabel('Rate (%)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# question 4
## Q4. Funnel Visualization with Storytelling Build Plotly funnel charts: an overall funnel and a segmented funnel (e.g., by trip length or driver rating). Use color gradients to highlight drop severity and add annotations for key insights. Write a detailed summary interpreting the funnel visuals.

In [ ]:
num_search_requests = dfs['search_data']['search_request_id'].nunique()
num_quotes = dfs['quote_data']['quote_id'].nunique()
num_bookings = dfs['booking_data']['booking_id'].nunique()
num_completed_bookings = dfs['booking_data'][dfs['booking_data']['status'] == 'COMPLETED']['booking_id'].nunique()

overall_funnel_df = pd.DataFrame({
    'Stage': ['Search Requests', 'Quotes', 'Bookings', 'Completed Bookings'],
    'Count': [num_search_requests, num_quotes, num_bookings, num_completed_bookings]
})

print("Overall Booking Funnel:")
display(overall_funnel_df)
import plotly.graph_objects as go

fig = go.Figure(go.Funnel(
    y = overall_funnel_df['Stage'],
    x = overall_funnel_df['Count'],
    textinfo = "value+percent initial",
    marker = {"color": ["deepskyblue", "lightsalmon", "tan", "teal"]},
    connector = {"line": {"color": "royalblue", "dash": "dot", "width": 3}}
))

fig.update_layout(
    title_text='Overall Booking Funnel',
    showlegend=False
)

fig.show()

booking_quote_merge = pd.merge(dfs['booking_data'], dfs['quote_data'][['quote_id', 'search_request_id']], on='quote_id', how='left')
full_funnel_data = pd.merge(booking_quote_merge, dfs['search_data'][['search_request_id', 'estimated_distance']], on='search_request_id', how='left')

full_funnel_data['stage'] = 'Bookings'
full_funnel_data.loc[full_funnel_data['status'] == 'COMPLETED', 'stage'] = 'Completed Bookings'

searches_only = dfs['search_data'][~dfs['search_data']['search_request_id'].isin(full_funnel_data['search_request_id'])]
searches_only['stage'] = 'Search Requests'
searches_only['booking_id'] = None
searches_only['quote_id'] = None


quotes_only = dfs['quote_data'][~dfs['quote_data']['quote_id'].isin(full_funnel_data['quote_id'])]
quotes_only = pd.merge(quotes_only, dfs['search_data'][['search_request_id', 'estimated_distance']], on='search_request_id', how='left')
quotes_only['stage'] = 'Quotes'
quotes_only['booking_id'] = None

full_funnel_data_subset = full_funnel_data[['search_request_id', 'estimated_distance', 'stage', 'booking_id', 'quote_id']]
searches_only_subset = searches_only[['search_request_id', 'estimated_distance', 'stage', 'booking_id', 'quote_id']]
quotes_only_subset = quotes_only[['search_request_id', 'estimated_distance', 'stage', 'booking_id', 'quote_id']]


segmented_funnel_data = pd.concat([full_funnel_data_subset, searches_only_subset, quotes_only_subset])

trip_distance_bins = [0, 5000, 10000, 25000, 50000, 100000, float('inf')]
trip_distance_labels = ['0-5km', '5-10km', '10-25km', '25-50km', '50-100km', '>100km']
segmented_funnel_data['trip_distance_bucket'] = pd.cut(segmented_funnel_data['estimated_distance'], bins=trip_distance_bins, labels=trip_distance_labels, right=False)

stage_order = ['Search Requests', 'Quotes', 'Bookings', 'Completed Bookings']


segmented_funnel_counts_list = []

for bucket in trip_distance_labels:
    bucket_data = segmented_funnel_data[segmented_funnel_data['trip_distance_bucket'] == bucket]
    for stage in stage_order:
        count = 0
        if stage == 'Search Requests':
            count = bucket_data[bucket_data['stage'] == stage]['search_request_id'].nunique()
        elif stage == 'Quotes':
            count = bucket_data[bucket_data['stage'] == stage]['quote_id'].nunique()
        elif stage == 'Bookings' or stage == 'Completed Bookings':
             count = bucket_data[bucket_data['stage'] == stage]['booking_id'].nunique()

        segmented_funnel_counts_list.append({'trip_distance_bucket': bucket, 'stage': stage, 'count': count})

segmented_funnel_counts = pd.DataFrame(segmented_funnel_counts_list)

segmented_funnel_counts['stage'] = pd.Categorical(segmented_funnel_counts['stage'], categories=stage_order, ordered=True)
segmented_funnel_counts = segmented_funnel_counts.sort_values(['trip_distance_bucket', 'stage'])

print("Segmented Funnel Data by Trip Distance Bucket:")
display(segmented_funnel_counts.head())
import plotly.graph_objects as go

funnel_charts = []

for bucket in segmented_funnel_counts['trip_distance_bucket'].unique():
    bucket_data = segmented_funnel_counts[segmented_funnel_counts['trip_distance_bucket'] == bucket]

    funnel = go.Funnel(
        name=bucket,
        y=bucket_data['stage'],
        x=bucket_data['count'],
        textinfo="value+percent initial",
        marker={"color": ["deepskyblue", "lightsalmon", "tan", "teal"]},
        connector={"line": {"color": "royalblue", "dash": "dot", "width": 3}}
    )

    funnel_charts.append(funnel)

fig = go.Figure(data=funnel_charts)

fig.update_layout(
    title_text='Segmented Booking Funnel by Trip Distance',
    height=600,
)

fig.show()